# Room Control System

![](images/room_clip.png)

---

## Aim

This application notebook uses Grove sensors and actuators to control a room.

## References 
* [Grove temperature](https://www.seeedstudio.com/Grove-Temperature-Sensor.html) 
* [Grove Servo](https://wiki.seeedstudio.com/Grove-Servo.html)  
* [Grove Relay](https://www.seeedstudio.com/Grove-Relay.html)  
* [Grove I2C OLED](https://wiki.seeedstudio.com/Grove-OLED_Display_0.96inch/) 
* [Grove RGB LED Stick](https://www.seeedstudio.com/Grove-RGB-LED-Stick-10-WS2813-Mini.html) 
* [Grove Slide Potentiometer](https://www.seeedstudio.com/Grove-Slide-Potentiometer.html) 
* [Grove Base Shield V2.0](https://www.seeedstudio.com/Base-Shield-V2.html) 
* [PYNQ Asyncio](https://pynq.readthedocs.io/en/latest/overlay_design_methodology/pynq_and_asyncio.html?highlight=asyncio)
* [Color chart](https://www.color-blindness.com/color-name-hue/)

## Last revised
* 22 April 2021
    + Initial version
---

## Load _base_ Overlay

In [1]:
from time import sleep

from IPython.display import display, HTML
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter

base = BaseOverlay('base.bit')

## Constructing application with Grove Base Shield V2.0 (Arduino)

<div class="alert alert-box alert-danger">
    <h4 class="alert-heading">Stop before proceeding further! </h4>
     Please make sure you had connected all the sensors before switching the board ON. If the sensors are connected while the board is ON it can cause the system to shutdown. For safe use, you can note down the connections shown below, switch the board OFF make the physical connections and the switch the board back ON.
    
   <h4 class="alert-heading">Make Physical Connections </h4>
    <ul>
        <li>Insert the Grove Base Shield into the Arduino connector on the board.</li>
        <li>Connect the grove_potentiometer module to A3 connector of the Grove Base Shield.</li>
        <li>Connect the grove_temperature module to A1 connector of the Grove Base Shield.</li>
        <li>Connect the grove_servo module to D5 connector of the Grove Base Shield.</li>
        <li>Connect the grove_relay module to D6 connector of the Grove Base Shield.</li>
        <li>Connect the grove_led_stick module to D7 connector of the Grove Base Shield.</li>
        <li>Connect the grove_oled module to any of the I2C connectors of the Grove Base Shield.</li></ul>

</div>

![](images/room_control.png)

### Adapter configuration

In [2]:
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, 
                                   A3='grove_potentiometer', A1='grove_temperature', 
                                   D5='grove_servo', D6='grove_relay', D7='grove_led_stick',
                                   I2C='grove_oled')

### Define device objects

In [3]:
blinds_slider = adapter.A3
blinds_servo = adapter.D5
temp_sensor = adapter.A1
led_stick = adapter.D7
light_relay = adapter.D6
display = adapter.I2C

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Notes before running the next cell </h4>
    <ul>
        <li><b>Light control </b></li>
        <ul><li>Toggle Switch SW0 on the board to turn the relay light ON or OFF.</li></ul>
        <li><b>Blinds control </b></li>
        <ul><li>Toggle Switch SW1 on the board to control the servo, switching ON will turn ON the Grove slider function, switching OFF will overwrite the Grove slider and turn off the servo</li></ul>
        <li><b>Mood lights color control </b></li>
        <ul><li> Press BTN0 button to scan through different colors of the led stick.</li></ul>
        <ul><li>Press BTN1 button to turn OFF the led stick</li></ul>
        <li><b>Exit</b></li>
        <ul><li>Press BTN3 button to exit the application</li></ul>
    </ul>
</div>

![](images/rcs_ctl.png)

### Basic loop implementation

In [19]:
led_colors = {0xff: 'Blue', 0xe4ff: 'Aqua', 0xff5c: 'Spring', 0xff10: 'Lime',
              0x3eff00: 'Harlequin', 0xfffa00: 'Yellow', 0xffc800: 'Tangerine',
              0xff4600: 'Orange', 0xff2800: 'Scarlet', 0xff0000: 'Red'}

color_keys = list(led_colors.keys())


def set_leds(color):
    for i in range(10):
        led_stick.set_pixel(i, color)


blinds_position = 0
count = 0
state = None
set_leds(color_keys[0])
led_stick.show()
display.clear_display()
display.set_next_row_wrap_mode()

try:
    while base.buttons[3].read() == 0:
        if base.switches[1].read() == 1:
            blinds_position = blinds_slider.position * 180
            blinds_servo.set_angular_position(blinds_position)
        else:
            blinds_position = 0
            blinds_servo.set_angular_position(blinds_position)

        if base.switches[0].read() == 1:
            light_relay.on()
        else:
            light_relay.off()

        if base.buttons[0].read() == 1:
            state = None
            if count < 9:
                count += 1
            else:
                count = 0
            set_leds(color_keys[count])
            led_stick.show()
        elif base.buttons[1].read() == 1:
            led_stick.clear()
            state = 'clear'

        display.set_position(0, 0)
        display.put_string(f'Temp: ')
        display.set_position(0, 6)
        display.put_string(f'{temp_sensor.temperature*9/5+32:.1f} F')
        display.set_position(2, 0)
        display.put_string('Blinds are down' if (
                blinds_position <= 90) else 'Blinds are up  ')
        display.set_position(4, 0)
        display.put_string('Lights are ON ' if (
                base.switches[0].read() == 1) else 'Lights are OFF')
        display.set_position(6, 0)
        display.put_string(f'Mood light color: None         ' 
                           if (base.buttons[1].read() == 1 or state == 'clear')
                           else f'Mood light color: {led_colors[color_keys[count]]}     ')

    display.clear_display()
    led_stick.clear()
    blinds_servo.set_angular_position(0)
    light_relay.off()
except KeyboardInterrupt:
    print('Kernel Interrupted')


---

## Asyncio Coroutine Implementation

The Basic loop implementation works well as the temperature sensor is queried in every iteration while the servo, relay and led_stick modules respond to actuation. All this happens in less than a second.   
However, if the application requires the temperature sensor to be queried every 10 seconds or so, the servo, relay and led_stick modules will have to wait for the temperature sensor to complete and therefore cannot respond.   
e.g.
```Python
try:
    while base.buttons[3].read() == 0:
        
        ...
        
        display.put_string(f'{temp_sensor.temperature*9/5+32:.1f} F')
        
        sleep(10)
        
        ...
        
        # Code for servo, relay and led_stick control
        
        ...

        # Display status for servo, relay and led_stick
        # Clear Grove modules
        
        ...
        
except KeyboardInterrupt:
    print('Stopping the loop')
```

To solve this problem we use the Asyncio coroutine implementation   
   
> The Python asyncio library manages multiple IO-bound tasks asynchronously, thereby avoiding any blocking caused by waiting for responses from slower IO subsystems. Instead, the program can continue to execute other tasks that are ready to run. When the previously-busy tasks are ready to resume, they will be executed in turn, and the cycle is repeated.   

Asyncio coroutine implementation is shown below

In [12]:
import asyncio

blinds_position = 0
state = None
temp = '32 F'
count = 0

led_colors = {0xff: 'Blue', 0xe4ff: 'Aqua', 0xff5c: 'Spring', 0xff10: 'Lime',
              0x3eff00: 'Harlequin', 0xfffa00: 'Yellow', 0xffc800: 'Tangerine',
              0xff4600: 'Orange', 0xff2800: 'Scarlet', 0xff0000: 'Red'}

color_keys = list(led_colors.keys())


def set_leds(color):
    for i in range(10):
        led_stick.set_pixel(i, color)

### Temperature coroutine

In [13]:
@asyncio.coroutine
def temp_monitor_display(period):
    global led_colors
    global temp
    
    while True:
        temp = f'{temp_sensor.temperature*9/5+32:.1f} F'
        yield from asyncio.sleep(period)

### Blinds servo control coroutine

In [14]:
@asyncio.coroutine
def blinds_controller(period):
    global blinds_position
    
    while True:
        # Manual overwrite with Grove slider if switch 1 is on
        if base.switches[1].read() == 1:
            blinds_position = blinds_slider.position * 180
            blinds_servo.set_angular_position(blinds_position)
        else:
            blinds_position = 0
            blinds_servo.set_angular_position(blinds_position)
        yield from asyncio.sleep(period)

### Mood lights - Led stick coroutine

In [15]:
@asyncio.coroutine
def mood_leds(period):
    global count
    global color_keys
    global state
    
    set_leds(color_keys[0])
    led_stick.show()
    
    while True:
        if base.buttons[0].read() == 1:
            state = None
            if count < 9:
                count += 1
            else:
                count = 0
            set_leds(color_keys[count])
            led_stick.show()
        elif base.buttons[1].read() == 1:
            led_stick.clear()
            state = 'clear'
        yield from asyncio.sleep(period)

### Light relay control and OLED display coroutine

In [16]:
@asyncio.coroutine
def lights(period):
    display.clear_display()
    display.set_next_row_wrap_mode()
    
    while True:
        if base.switches[0].read() == 1:
            light_relay.on()
        else:
            light_relay.off()
            
        display.set_position(0, 0)
        display.put_string(f'Temp: ')
        display.set_position(0, 6)
        display.put_string(temp)
        display.set_position(2, 0)
        display.put_string('Blinds are down' if (
                    blinds_position <= 90) else 'Blinds are up  ')
        display.set_position(4, 0)
        display.put_string('Lights are ON ' if (
                    base.switches[0].read() == 1) else 'Lights are OFF')
        display.set_position(6, 0)
        display.put_string(f'Mood light color: None         ' if (
                base.buttons[1].read() == 1 or state == 'clear') else 
                f'Mood light color:{led_colors[color_keys[count]]}     ')
        yield from asyncio.sleep(period)

### Create async tasks
Temperature sensor is queried every 10 seconds while other modules are controlled multiple times every second

In [17]:
tasks = [asyncio.ensure_future(temp_monitor_display(10)),
         asyncio.ensure_future(lights(0.3)),
         asyncio.ensure_future(mood_leds(0.2)),
         asyncio.ensure_future(blinds_controller(0.01))]

<div class="alert alert-box alert-info">
   <h4 class="alert-heading">Notes before running the next cell </h4>
    <ul>
        <li><b>Light control </b></li>
        <ul><li>Toggle Switch SW0 on the board to turn the relay light ON or OFF.</li></ul>
        <li><b>Blinds control </b></li>
        <ul><li>Toggle Switch SW1 on the board to control the servo, switching ON will turn ON the Grove slider function, switching OFF will overwrite the Grove slider and turn off the servo</li></ul>
        <li><b>Mood lights color control </b></li>
        <ul><li> Press BTN0 button to scan through different colors of the led stick.</li></ul>
        <ul><li>Press BTN1 button to turn OFF the led stick</li></ul>
        <li><b>Exit</b></li>
        <ul><li>Press BTN3 button to exit the application</li></ul>
    </ul>
</div>

![](images/rcs_ctl.png)

### Execute tasks and clear modules when tasks are cancelled

In [18]:
base.buttons[3].wait_for_value(1)
[t.cancel() for  t in tasks]
display.clear_display()
led_stick.clear()
blinds_servo.set_angular_position(0)
light_relay.off()

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----